In [104]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [105]:
df_regular_21 = pd.read_csv('2021-2022 NBA Player Stats - Regular.csv',sep=";", encoding="Windows-1252")

In [106]:
df_regular_22 = pd.read_csv('2022-2023 NBA Player Stats - Regular.csv',sep=";", encoding="Windows-1252")

TOT means the total of the stats for the relevant season. If a player changes teams in a season, there will be a TOT row in the Team column.
Rk is index for each row.

In [107]:
df_regular_21.drop(columns=['Rk'], inplace=True)
df_regular_21 = df_regular_21.loc[df_regular_21['Tm'] != 'TOT']

(715, 29)

In [113]:
unique_team_21 = df_regular_21['Tm'].sort_values().unique()
print(unique_team_21)

['ATL' 'BOS' 'BRK' 'CHI' 'CHO' 'CLE' 'DAL' 'DEN' 'DET' 'GSW' 'HOU' 'IND'
 'LAC' 'LAL' 'MEM' 'MIA' 'MIL' 'MIN' 'NOP' 'NYK' 'OKC' 'ORL' 'PHI' 'PHO'
 'POR' 'SAC' 'SAS' 'TOR' 'UTA' 'WAS']


In [108]:
df_regular_22.drop(columns=['Rk'], inplace=True)
df_regular_22 = df_regular_22.loc[df_regular_22['Tm'] != 'TOT']

(609, 29)

These are the 30 teams

In [ ]:
Assumption: We will combine 2 years of data, so we have 60 rows in total.
    Each team plays 82 games with the rest 29 teams in the whole regular season. 
    Success of team is defined by total number of wins. This number is manually keyed in from NBA website.
Hypothesis: Teams with more star players, strong defense and efficient offense are more likely to win.
Star player is at player level, and defense and offense is at team level. 
To calculate team level data, we can sum all the goals from all players and then divide by 82 to get team performance.
    Start player is defined as points per game >18 or total rebound per game >10 or assist per game >10
    Defense is defined as total rebounds per game, total blocks per game and total steals per game
    Offense is definded as total assists per game, total 3-point field goals per game
    field goal percentage = total field goal/total field goal attempts from all players
    

Add total rebounds,blocks, steals,assists,3-point field goals,field goals, field goal attempts  of each player as new columns
Total rebound = games played * rebound per game, round up to nearest integer

In [123]:
df_regular_21['TRBD']= round(df_regular_21['TRB']* df_regular_21['G'])
df_regular_21['TBLK']= round(df_regular_21['BLK']* df_regular_21['G'])
df_regular_21['TAST']= round(df_regular_21['AST']* df_regular_21['G'])
df_regular_21['TSTL']= round(df_regular_21['STL']* df_regular_21['G'])
df_regular_21['T3P']= round(df_regular_21['3P']* df_regular_21['G'])
df_regular_21['TFG']= round(df_regular_21['FG']* df_regular_21['G'])
df_regular_21['TFGA']= round(df_regular_21['FGA']* df_regular_21['G'])

In [124]:
df_regular_21.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV,PF,PTS,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA
0,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,...,1.2,2.1,9.1,474.0,44.0,80.0,36.0,58.0,263.0,606.0
1,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,...,1.5,2.0,6.9,760.0,61.0,258.0,68.0,0.0,213.0,388.0
2,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,...,2.6,3.1,19.1,566.0,45.0,190.0,78.0,0.0,409.0,728.0
3,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,...,0.5,1.1,4.1,86.0,10.0,22.0,6.0,6.0,54.0,131.0
4,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,0.550,...,0.9,1.7,12.9,258.0,47.0,42.0,14.0,14.0,254.0,456.0


In [141]:
summary_21 = df_regular_21.groupby(['Tm'])[['TRBD','TBLK','TAST','TSTL','T3P']].sum()/82

,TRBD,TBLK,TAST,TSTL,T3P
Tm,,,,,
ATL,44.109756,4.329268,24.500000,6.975610,13.024390
BOS,46.085366,5.853659,24.865854,7.292683,13.317073
BRK,44.487805,5.414634,25.402439,7.170732,11.487805
CHI,42.402439,4.134146,23.792683,7.121951,10.634146
CHO,44.560976,4.768293,28.195122,8.548780,13.865854


In [142]:
fgpct_21 = df_regular_21.groupby(['Tm'])[['TFG','TFGA']].sum()
fgpct_21['FG%'] = fgpct_21['TFG']/fgpct_21['TFGA']

,TFG,TFGA,FG%
Tm,,,
ATL,3403.0,7237.0,0.470222
BOS,3345.0,7161.0,0.467114
BRK,3453.0,7248.0,0.476407
CHI,3427.0,7120.0,0.481320
CHO,3511.0,7507.0,0.467697


In [143]:
summary_21 = summary_21.join(fgpct_21,on = 'Tm',how='left')
summary_21.head()

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%
Tm,,,,,,,,
ATL,44.109756,4.329268,24.500000,6.975610,13.024390,3403.0,7237.0,0.470222
BOS,46.085366,5.853659,24.865854,7.292683,13.317073,3345.0,7161.0,0.467114
BRK,44.487805,5.414634,25.402439,7.170732,11.487805,3453.0,7248.0,0.476407
CHI,42.402439,4.134146,23.792683,7.121951,10.634146,3427.0,7120.0,0.481320
CHO,44.560976,4.768293,28.195122,8.548780,13.865854,3511.0,7507.0,0.467697


In [144]:
df_regular_22['TRBD']= round(df_regular_22['TRB']* df_regular_22['G'])
df_regular_22['TBLK']= round(df_regular_22['BLK']* df_regular_22['G'])
df_regular_22['TAST']= round(df_regular_22['AST']* df_regular_22['G'])
df_regular_22['TSTL']= round(df_regular_22['STL']* df_regular_22['G'])
df_regular_22['T3P']= round(df_regular_22['3P']* df_regular_22['G'])
df_regular_22['TFG']= round(df_regular_22['FG']* df_regular_22['G'])
df_regular_22['TFGA']= round(df_regular_22['FGA']* df_regular_22['G'])
summary_22 = df_regular_22.groupby(['Tm'])[['TRBD','TBLK','TAST','TSTL','T3P']].sum()/82
fgpct_22 = df_regular_22.groupby(['Tm'])[['TFG','TFGA']].sum()
fgpct_22['FG%'] = fgpct_22['TFG']/fgpct_22['TFGA']
summary_22 = summary_22.join(fgpct_22,on = 'Tm',how='left')
summary_22.head()

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%
Tm,,,,,,,,
ATL,44.378049,4.865854,25.085366,7.097561,10.829268,3677.0,7570.0,0.485733
BOS,45.463415,5.378049,26.719512,6.487805,16.097561,3456.0,7281.0,0.474660
BRK,40.560976,6.207317,25.573171,7.304878,12.634146,3393.0,6973.0,0.486591
CHI,42.451220,4.560976,24.414634,7.902439,10.304878,3486.0,7119.0,0.489676
CHO,44.536585,5.182927,25.121951,7.743902,10.731707,3369.0,7416.0,0.454288


If a player meets any of the following three stats, we encode him as a star player:
points per game >18 or
total rebound per game >10 or
assist per game >10

In [145]:
def star_player(row):
    if row['PTS']>=18:
        return 1
    if row['TRB']>=10:
        return 1
    if row['AST']>=10:
        return 1
    else:
        return 0

In [147]:
df_regular_21['Star'] = df_regular_21.apply(star_player, axis=1)
df_regular_21.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,PF,PTS,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,Star
0,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,...,2.1,9.1,474.0,44.0,80.0,36.0,58.0,263.0,606.0,0
1,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,...,2.0,6.9,760.0,61.0,258.0,68.0,0.0,213.0,388.0,0
2,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,...,3.1,19.1,566.0,45.0,190.0,78.0,0.0,409.0,728.0,1
3,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,...,1.1,4.1,86.0,10.0,22.0,6.0,6.0,54.0,131.0,0
4,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,0.550,...,1.7,12.9,258.0,47.0,42.0,14.0,14.0,254.0,456.0,0


In [149]:
star_21 = df_regular_21.groupby(['Tm'])[['Star']].sum()
star_21.head()

,Star
Tm,
ATL,2
BOS,2
BRK,4
CHI,3
CHO,3


In [159]:
summary_21 = summary_21.join(star_21,on = 'Tm',how='left')
summary_21.reset_index()

ValueError: columns overlap but no suffix specified: Index(['Star'], dtype='object')

In [151]:
df_regular_22['Star'] = df_regular_22.apply(star_player, axis=1)
star_22 = df_regular_22.groupby(['Tm'])[['Star']].sum()
summary_22 = summary_22.join(star_22,on = 'Tm',how='left')
summary_22.head()

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star
Tm,,,,,,,,,
ATL,44.378049,4.865854,25.085366,7.097561,10.829268,3677.0,7570.0,0.485733,3
BOS,45.463415,5.378049,26.719512,6.487805,16.097561,3456.0,7281.0,0.474660,2
BRK,40.560976,6.207317,25.573171,7.304878,12.634146,3393.0,6973.0,0.486591,3
CHI,42.451220,4.560976,24.414634,7.902439,10.304878,3486.0,7119.0,0.489676,3
CHO,44.536585,5.182927,25.121951,7.743902,10.731707,3369.0,7416.0,0.454288,3


Add total wins each season for each team

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star
Tm,,,,,,,,,
ATL,44.109756,4.329268,24.500000,6.975610,13.024390,3403.0,7237.0,0.470222,2
BOS,46.085366,5.853659,24.865854,7.292683,13.317073,3345.0,7161.0,0.467114,2
BRK,44.487805,5.414634,25.402439,7.170732,11.487805,3453.0,7248.0,0.476407,4
CHI,42.402439,4.134146,23.792683,7.121951,10.634146,3427.0,7120.0,0.481320,3
CHO,44.560976,4.768293,28.195122,8.548780,13.865854,3511.0,7507.0,0.467697,3


In [100]:
df = pd.concat([df_regular_21,df_regular_22])

In [152]:
wins = pd.read_csv('NBA Standings.csv',sep=",")

In [153]:
wins.head()

,Tm,Wins_21,Wins_22
0,ATL,43,41
1,BOS,51,57
2,BRK,44,45
3,CHO,43,27
4,CHI,46,51


In [162]:
summary_21= summary_21.reset_index()

In [163]:
summary_21 = summary_21.merge(wins[['Tm','Wins_21']],on = 'Tm',how='left')

In [166]:
summary_22.head()

,Tm,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star,Wins_22
0,ATL,44.378049,4.865854,25.085366,7.097561,10.829268,3677.0,7570.0,0.485733,3,41
1,BOS,45.463415,5.378049,26.719512,6.487805,16.097561,3456.0,7281.0,0.474660,2,57
2,BRK,40.560976,6.207317,25.573171,7.304878,12.634146,3393.0,6973.0,0.486591,3,45
3,CHI,42.451220,4.560976,24.414634,7.902439,10.304878,3486.0,7119.0,0.489676,3,51
4,CHO,44.536585,5.182927,25.121951,7.743902,10.731707,3369.0,7416.0,0.454288,3,27


In [165]:
summary_22= summary_22.reset_index()
summary_22 = summary_22.merge(wins[['Tm','Wins_22']],on = 'Tm',how='left')

In [170]:
df= pd.concat([summary_21, summary_22])
df.shape

(60, 12)

In [171]:
df.head()

,Tm,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star,Wins_21,Wins_22
0,ATL,44.109756,4.329268,24.500000,6.975610,13.024390,3403.0,7237.0,0.470222,2,43.0,NaN
1,BOS,46.085366,5.853659,24.865854,7.292683,13.317073,3345.0,7161.0,0.467114,2,51.0,NaN
2,BRK,44.487805,5.414634,25.402439,7.170732,11.487805,3453.0,7248.0,0.476407,4,44.0,NaN
3,CHI,42.402439,4.134146,23.792683,7.121951,10.634146,3427.0,7120.0,0.481320,3,46.0,NaN
4,CHO,44.560976,4.768293,28.195122,8.548780,13.865854,3511.0,7507.0,0.467697,3,43.0,NaN


In [178]:
def ifnull(row):
  if row['Wins_21'] =='NaN':
    return row['Wins_22']
  else:
    return row['Wins_21']

In [187]:
df['Win'] = df['Wins_21']
df['Win'].fillna(df['Wins_22'], inplace=True)
df

,Tm,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star,Wins_21,Wins_22,Win
0,ATL,44.109756,4.329268,24.500000,6.975610,13.024390,3403.0,7237.0,0.470222,2,43.0,NaN,43.0
1,BOS,46.085366,5.853659,24.865854,7.292683,13.317073,3345.0,7161.0,0.467114,2,51.0,NaN,51.0
2,BRK,44.487805,5.414634,25.402439,7.170732,11.487805,3453.0,7248.0,0.476407,4,44.0,NaN,44.0
3,CHI,42.402439,4.134146,23.792683,7.121951,10.634146,3427.0,7120.0,0.481320,3,46.0,NaN,46.0
4,CHO,44.560976,4.768293,28.195122,8.548780,13.865854,3511.0,7507.0,0.467697,3,43.0,NaN,43.0
5,CLE,44.243902,4.158537,25.097561,7.097561,11.512195,3250.0,6945.0,0.467963,2,44.0,NaN,44.0
6,DAL,42.890244,3.963415,23.463415,6.853659,13.195122,3226.0,6985.0,0.461847,2,52.0,NaN,52.0
7,DEN,44.207317,3.853659,27.804878,7.268293,12.780488,3410.0,7080.0,0.481638,1,48.0,NaN,48.0
8,DET,43.085366,4.829268,23.451220,7.585366,11.304878,3127.0,7275.0,0.429828,1,23.0,NaN,23.0
9,GSW,45.365854,4.548780,27.048780,8.756098,14.512195,3326.0,7082.0,0.469641,3,53.0,NaN,53.0


Correlation analysis

,Tm,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star,Wins_21,Wins_22,Win
0,ATL,44.109756,4.329268,24.500000,6.975610,13.024390,3403.0,7237.0,0.470222,2,43.0,NaN,43.0
1,BOS,46.085366,5.853659,24.865854,7.292683,13.317073,3345.0,7161.0,0.467114,2,51.0,NaN,51.0
2,BRK,44.487805,5.414634,25.402439,7.170732,11.487805,3453.0,7248.0,0.476407,4,44.0,NaN,44.0
3,CHI,42.402439,4.134146,23.792683,7.121951,10.634146,3427.0,7120.0,0.481320,3,46.0,NaN,46.0
4,CHO,44.560976,4.768293,28.195122,8.548780,13.865854,3511.0,7507.0,0.467697,3,43.0,NaN,43.0
5,CLE,44.243902,4.158537,25.097561,7.097561,11.512195,3250.0,6945.0,0.467963,2,44.0,NaN,44.0
6,DAL,42.890244,3.963415,23.463415,6.853659,13.195122,3226.0,6985.0,0.461847,2,52.0,NaN,52.0
7,DEN,44.207317,3.853659,27.804878,7.268293,12.780488,3410.0,7080.0,0.481638,1,48.0,NaN,48.0
8,DET,43.085366,4.829268,23.451220,7.585366,11.304878,3127.0,7275.0,0.429828,1,23.0,NaN,23.0
9,GSW,45.365854,4.548780,27.048780,8.756098,14.512195,3326.0,7082.0,0.469641,3,53.0,NaN,53.0


Linear regression

In [182]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [184]:
y = df['Win']
x = df[['TRBD','TBLK','TAST','TSTL','T3P','FG%','Star']]
model = sm.OLS(y,x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    Win   R-squared (uncentered):                   0.950
Model:                            OLS   Adj. R-squared (uncentered):              0.944
Method:                 Least Squares   F-statistic:                              145.2
Date:                Sun, 10 Dec 2023   Prob (F-statistic):                    2.98e-32
Time:                        23:18:26   Log-Likelihood:                         -219.77
No. Observations:                  60   AIC:                                      453.5
Df Residuals:                      53   BIC:                                      468.2
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TRBD          -0.4272      0.565     -0.756      0.453      -1.561       0.707
TBLK          -0.7059      1.950     -0.362      0.719      -4.617       3.205
TAST           0.5004      0.929      0.539      0.592      -1.363       2.364
TSTL           1.3495      1.807      0.747      0.459      -2.275       4.974
T3P            1.6319      0.948      1.722      0.091      -0.269       3.533
FG%           32.3673     64.066      0.505      0.615     -96.132     160.866
Star           2.1331      1.590      1.341      0.186      -1.057       5.323
==============================================================================
Omnibus:                        1.407   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.495   Jarque-Bera (JB):                1.427
Skew:                          -0.306   Prob(JB):                        0.490
Kurtosis:                       2.556   Cond. No.                     2.62e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [189]:
df[['Win','TRBD','TBLK','TAST','TSTL','T3P','FG%','Star']].corr()

,Win,TRBD,TBLK,TAST,TSTL,T3P,FG%,Star
Win,1.000000,0.237501,0.054931,0.341840,0.212145,0.325804,0.541394,0.235286
TRBD,0.237501,1.000000,0.257230,0.085800,0.115797,0.083101,-0.229387,0.043659
TBLK,0.054931,0.257230,1.000000,0.101376,0.197005,-0.042024,-0.012076,0.098280
TAST,0.341840,0.085800,0.101376,1.000000,0.121808,0.218376,0.555078,0.005599
TSTL,0.212145,0.115797,0.197005,0.121808,1.000000,-0.201207,-0.139523,0.270665
T3P,0.325804,0.083101,-0.042024,0.218376,-0.201207,1.000000,0.055364,-0.012082
FG%,0.541394,-0.229387,-0.012076,0.555078,-0.139523,0.055364,1.000000,0.193046
Star,0.235286,0.043659,0.098280,0.005599,0.270665,-0.012082,0.193046,1.000000
